In [4]:

from entity_retrieval import surface_index_memory
from itertools import product
from SPARQLWrapper import SPARQLWrapper, JSON
from tqdm import tqdm
import ujson
import re

In [5]:
import nltk
# nltk.download('punkt_tab')

In [6]:
surface_index = surface_index_memory.EntitySurfaceIndexMemory(
    "vi_entity_list_file_wikidata_complete_all_mention", "vi_entity_surface_map_file_wikidata_complete_all_mention","vi_wiki_complete_all_mention" )

02/23/2025 22:00:01 - INFO - entity_retrieval.surface_index_memory -   Loading entity vocabulary from disk.
02/23/2025 22:00:01 - INFO - entity_retrieval.surface_index_memory -   Loading surfaces from disk.
02/23/2025 22:00:01 - INFO - entity_retrieval.surface_index_memory -   Done initializing surface index.


In [7]:
def is_valid_expression(expr):
    """Kiểm tra tính hợp lệ của biểu thức bằng cách đếm số ngoặc mở và đóng."""
    count = 0
    for char in expr:
        if char == '(':
            count += 1
        elif char == ')':
            count -= 1
        if count < 0:
            return False  # Gặp ngoặc đóng trước ngoặc mở
    return count == 0

def fix_unbalanced_parentheses(expr):
    """Loại bỏ ngoặc đóng dư nếu có."""
    while not is_valid_expression(expr) and expr.endswith(')'):
        expr = expr[:-1]
    return expr

In [8]:
import torch
from transformers import AutoTokenizer, AutoModel
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Load model & tokenizer
tokenizer = AutoTokenizer.from_pretrained("VoVanPhuc/sup-SimCSE-VietNamese-phobert-base")
model = AutoModel.from_pretrained("VoVanPhuc/sup-SimCSE-VietNamese-phobert-base")


def get_embedding(text):
    """Trích xuất vector embedding từ mô hình"""
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=128)
    with torch.no_grad():
        outputs = model(**inputs)

    # Dùng mean pooling thay vì pooler_output
    embeddings = outputs.last_hidden_state.mean(dim=1).cpu().numpy()
    return embeddings

def find_entity_or_relation(label, label_map, facc1_index, top_k=50, similarity_threshold=0.2):
    """
    Tìm thực thể hoặc quan hệ từ gold maps, SimCSE hoặc FACC1.
    """
    label_lower = label.lower()

    # Nếu có trong label_map, trả về ngay
    if label_map and label_lower in label_map:
        return label_map[label_lower]

    # Lấy embedding cho label
    label_embedding = get_embedding(label_lower).reshape(1, -1)

    if label_map:
        label_keys = list(label_map.keys())
        label_embeddings = np.array([get_embedding(k) for k in label_keys]).squeeze(1)

        # Đảm bảo đúng shape
        if len(label_embeddings.shape) == 1:
            label_embeddings = label_embeddings.reshape(1, -1)

        # Tính cosine similarity
        similarities = cosine_similarity(label_embedding, label_embeddings).flatten()

        # Chọn thực thể gần nhất
        merged_list = list(zip(label_keys, similarities))
        sorted_list = sorted(merged_list, key=lambda x: x[1], reverse=True)

        if sorted_list and sorted_list[0][1] > similarity_threshold:
            return label_map[sorted_list[0][0]]

    # Nếu không tìm thấy, thử trong KB (FACC1)
    facc1_cand_entities = facc1_index.get_indexrange_entity_el_pro_one_mention(label_lower, top_k=top_k)
    if facc1_cand_entities:
        best_match = max(facc1_cand_entities.items(), key=lambda x: x[1])
        return best_match[0]

    return label  # Trả về label nếu không tìm thấy

def parse_nsexpr(expr):
    """
    Chuyển chuỗi biểu thức thành cây cấu trúc dạng nested list.
    Hàm này dùng duyệt ký tự, khi gặp '(' sẽ tìm phần con cho đến khi khớp với ')',
    và giữ nguyên nội dung trong ngoặc vuông.
    """
    tokens = []
    i = 0
    while i < len(expr):
        if expr[i].isspace():
            i += 1
        elif expr[i] == '(':
            # Tìm phần con của biểu thức trong ngoặc đơn
            count = 1
            j = i + 1
            while j < len(expr) and count > 0:
                if expr[j] == '(':
                    count += 1
                elif expr[j] == ')':
                    count -= 1
                j += 1
            # Đệ quy phân tích phần con (loại bỏ ngoặc bao ngoài)
            subtree = parse_nsexpr(expr[i+1:j-1])
            tokens.append(subtree)
            i = j
        elif expr[i] == '[':
            # Giữ nguyên nội dung trong ngoặc vuông
            j = expr.find(']', i)
            if j == -1:
                return ""
                raise ValueError("Không tìm thấy dấu ']' kết thúc.")
                
            token = expr[i:j+1].strip()
            tokens.append(token)
            i = j + 1
        else:
            # Đọc một token cho đến khi gặp khoảng trắng hoặc ngoặc
            j = i
            while j < len(expr) and (not expr[j].isspace()) and expr[j] not in ['(', ')']:
                j += 1
            tokens.append(expr[i:j])
            i = j
    return tokens

def collect_labels(tree):
    """
    Duyệt cây cấu trúc (nested list) để thu thập các nhãn của quan hệ và thực thể.
    Giả sử:
      - Biểu thức JOIN có dạng: ["JOIN", relation_part, entity_part]
      - Phần relation_part: nếu là list và bắt đầu bằng "R", thì phần thứ hai chứa nhãn quan hệ (dạng "[ label ]"). 
        Nếu là chuỗi dạng "[ label ]" thì đó cũng là nhãn quan hệ.
      - Phần entity_part: nếu là chuỗi dạng "[ label ]" thì đó là nhãn thực thể, nếu là list thì xử lý đệ quy.
      - Biểu thức AND sẽ có nhiều biểu thức con.
    """
    relations = []
    entities = []
    
    if isinstance(tree, list) and tree:
        # Nếu token đầu tiên là JOIN hoặc AND
        op = tree[0]
        if isinstance(op, str):
            op_upper = op.upper()
        else:
            op_upper = ""
        
        if op_upper == "JOIN":
            # Xử lý phần quan hệ
            if len(tree) >= 2:
                rel_part = tree[1]
                # Nếu là list dạng [ "R", "[ label ]" ]
                if isinstance(rel_part, list) and len(rel_part) >= 2 and isinstance(rel_part[0], str) and rel_part[0].upper() == "R":
                    token = rel_part[1]
                    if isinstance(token, str) and token.startswith('[') and token.endswith(']'):
                        rel_label = token[1:-1].strip()
                        relations.append(rel_label)
                # Nếu là chuỗi dạng "[ label ]"
                elif isinstance(rel_part, str) and rel_part.startswith('[') and rel_part.endswith(']'):
                    rel_label = rel_part[1:-1].strip()
                    relations.append(rel_label)
                else:
                    # Nếu không đúng định dạng, duyệt đệ quy
                    sub_rel, sub_ent = collect_labels(rel_part)
                    relations.extend(sub_rel)
                    entities.extend(sub_ent)
            # Xử lý phần thực thể
            if len(tree) >= 3:
                ent_part = tree[2]
                if isinstance(ent_part, list):
                    sub_rel, sub_ent = collect_labels(ent_part)
                    relations.extend(sub_rel)
                    entities.extend(sub_ent)
                elif isinstance(ent_part, str) and ent_part.startswith('[') and ent_part.endswith(']'):
                    ent_label = ent_part[1:-1].strip()
                    entities.append(ent_label)
        elif op_upper == "AND":
            # Với AND, duyệt tất cả các phần con
            for sub in tree[1:]:
                sub_rel, sub_ent = collect_labels(sub)
                relations.extend(sub_rel)
                entities.extend(sub_ent)
        else:
            # Nếu không phải JOIN hay AND, duyệt tất cả các phần tử nếu chúng là list
            for elem in tree:
                if isinstance(elem, list):
                    sub_rel, sub_ent = collect_labels(elem)
                    relations.extend(sub_rel)
                    entities.extend(sub_ent)
    return relations, entities



def extract_entities_and_relations(normed_expr):

    if not normed_expr or len(normed_expr) == 0:  # Kiểm tra nếu normed_expr rỗng
        return [], []
    
    if normed_expr[0] != "(":
        return [], []
    
    tree = parse_nsexpr(normed_expr)
    if tree is None:
        return [], []  # Trả về danh sách rỗng nếu parse thất bại
    
    return collect_labels(tree)



In [9]:
def find_entity(label, label_map, facc1_index, top_k=20, similarity_threshold=0.2):
    """
    Tìm thực thể hoặc quan hệ từ gold maps, SimCSE hoặc FACC1.
    """
    label_lower = label.lower()

    # Nếu có trong label_map, trả về ngay
    if label_map and label_lower in label_map:
        return label_map[label_lower]

    # Lấy embedding cho label
    label_embedding = get_embedding(label_lower).reshape(1, -1)

    if label_map:
        label_keys = list(label_map.keys())
        label_embeddings = np.array([get_embedding(k) for k in label_keys]).squeeze(1)

        # Đảm bảo đúng shape
        if len(label_embeddings.shape) == 1:
            label_embeddings = label_embeddings.reshape(1, -1)

        # Tính cosine similarity
        similarities = cosine_similarity(label_embedding, label_embeddings).flatten()

        # Chọn thực thể gần nhất
        merged_list = list(zip(label_keys, similarities))
        sorted_list = sorted(merged_list, key=lambda x: x[1], reverse=True)

        if sorted_list and sorted_list[0][1] > similarity_threshold:
            return label_map[sorted_list[0][0]]

    # Nếu không tìm thấy, thử trong KB (FACC1)
    facc1_cand_entities = facc1_index.get_indexrange_entity_el_pro_one_mention(label_lower, top_k=top_k)
    if facc1_cand_entities:
        best_match = max(facc1_cand_entities.items(), key=lambda x: x[1])
        return best_match[0]

    return label  # Trả về label nếu không tìm thấy

In [10]:
def find_relation(label, label_map, relation_kb_map, facc1_index, top_k=20, similarity_threshold=0.2):
    """
    Tìm quan hệ từ gold maps, SimCSE hoặc FACC1.
    """
    label_lower = label.lower()

    # Nếu có trong label_map, trả về ngay
    if label_map and label_lower in label_map:
        return label_map[label_lower]

    # Lấy embedding cho label
    label_embedding = get_embedding(label_lower).reshape(1, -1)

    if label_map:
        label_keys = list(label_map.keys())
        label_embeddings = np.array([get_embedding(k) for k in label_keys]).squeeze(1)

        # Đảm bảo đúng shape
        if len(label_embeddings.shape) == 1:
            label_embeddings = label_embeddings.reshape(1, -1)

        # Tính cosine similarity
        similarities = cosine_similarity(label_embedding, label_embeddings).flatten()

        # Chọn thực thể gần nhất
        merged_list = list(zip(label_keys, similarities))
        sorted_list = sorted(merged_list, key=lambda x: x[1], reverse=True)

        if sorted_list and sorted_list[0][1] > similarity_threshold:
            return label_map[sorted_list[0][0]]
    if relation_kb_map:
        label_keys = list(relation_kb_map.keys())
        label_embeddings = np.array([get_embedding(k) for k in label_keys]).squeeze(1)

        # Đảm bảo đúng shape
        if len(label_embeddings.shape) == 1:
            label_embeddings = label_embeddings.reshape(1, -1)

        # Tính cosine similarity
        similarities = cosine_similarity(label_embedding, label_embeddings).flatten()

        # Chọn thực thể gần nhất
        merged_list = list(zip(label_keys, similarities))
        sorted_list = sorted(merged_list, key=lambda x: x[1], reverse=True)

        if sorted_list and sorted_list[0][1] > similarity_threshold:
            return relation_kb_map[sorted_list[0][0]]

    return label 

In [11]:
class SExpressionParser:
    def __init__(self):
        self.var_counter = 1  # Đếm số biến trung gian (?X1, ?X2, ...)

    def get_new_var(self):
        """Tạo biến trung gian mới."""
        var_name = f"?X{self.var_counter}"
        self.var_counter += 1
        return var_name

    def parse_s_expr(self, s_expr):
        """Chuyển đổi S-Expression thành danh sách lồng nhau."""
        s_expr = re.sub(r'\(', ' ( ', s_expr)
        s_expr = re.sub(r'\)', ' ) ', s_expr)
        tokens = s_expr.split()
        return self.build_tree(tokens)

    def build_tree(self, tokens):
        """Chuyển đổi danh sách token thành cây lồng nhau."""
        if not tokens:
            return None
        token = tokens.pop(0)
        if token == "(":
            sub_expr = []
            while tokens[0] != ")":
                sub_expr.append(self.build_tree(tokens))
            tokens.pop(0)  # Bỏ dấu ")"
            return sub_expr
        elif token == ")":
            raise ValueError("Unexpected ')'")
        else:
            return token

    def process_join(self, expr, target_var):
        """
        Xử lý JOIN, tạo triple SPARQL.
        """
        triples = []
        if not isinstance(expr, list):
            return expr, triples

        if expr[0] == "AND":
            # Xử lý từng JOIN trong AND riêng lẻ
            for sub_expr in expr[1:]:
                _, sub_triples = self.process_join(sub_expr, target_var)
                triples.extend(sub_triples)
            return target_var, triples

        if expr[0] == "JOIN":
            right_expr = expr[2]
            right_triples = []
            if isinstance(right_expr, list) and right_expr[0] == "JOIN":
                right_var, right_triples = self.process_join(right_expr, self.get_new_var())
            else: 
                right_var = right_expr
            
            # Xử lý nhánh trái
            left_expr = expr[1]
            if isinstance(left_expr, list) and left_expr[0] == "R":
                rel = left_expr[1]
                right = right_var
                left = target_var
                if right[0] != '?':
                    right = "wd:" + right
                if left[0] !='?':
                    left = "wd:" + left   
                triples.append([right, f"wdt:{rel}", left])
            else:
                right = right_var
                left = target_var
                if right[0] != '?':
                    right = "wd:" + right
                if left[0] !='?':
                    left = "wd:" + left   
                triples.append([left, f"wdt:{left_expr}", right])

            # Thêm các triples từ nhánh phải trước khi thêm triple chính
            triples = right_triples + triples
            return target_var, triples

        return expr, triples

    def s_expr_to_sparql(self, s_expr):
        """Chuyển đổi từ S-Expression sang SPARQL."""
        if s_expr.count("(") != s_expr.count(")"):
            return None
        if s_expr.count("[") != s_expr.count("]"):
            return None
        parsed_expr = self.parse_s_expr(s_expr)
        target_var = "?answer"
        final_var, triples = self.process_join(parsed_expr, target_var)

        sparql_body = "\n  ".join([" ".join(t) + " ." for t in triples])
        sparql_query = f"""PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
SELECT DISTINCT {target_var} WHERE {{ 
  {sparql_body}
}}"""
        return sparql_query


def execute_query_with_odbc(sparql_query):
    """
    Thực thi truy vấn SPARQL trên Wikidata endpoint và trả về kết quả.
    """
    ENDPOINT_URL = "https://query.wikidata.org/sparql"
    if sparql_query == None or sparql_query == []:
        return []
    sparql = SPARQLWrapper(ENDPOINT_URL)
    sparql.setQuery(str(sparql_query))  # Ép kiểu về str cho chắc chắn
    sparql.setReturnFormat(JSON)
    
    try:
        response = sparql.query().convert()
        answers = [item["answer"]["value"] for item in response["results"]["bindings"] if "answer" in item]
        return answers
    except Exception as e:
        return []



In [12]:
def convert_normed_to_s_expression(normed_expr, gold_relation_map, gold_entity_map, relation_KB_map, facc1_index):
    """
    Chuyển đổi từ normed_sexpression sang s_expression.
    Sau khi trích xuất các nhãn quan hệ và thực thể từ normed_expr,
    ta lấy danh sách các mã ứng viên cho mỗi nhãn và tạo hoán vị giữa các cặp ứng viên đó.
    Kết quả trả về là một danh sách các s_expression khả dĩ.
    """
    normed_expr = fix_unbalanced_parentheses(normed_expr)
    # Trích xuất các nhãn quan hệ và thực thể từ biểu thức
    relations, entities = extract_entities_and_relations(normed_expr)
    
    # Tạo mapping từ token xuất hiện trong biểu thức sang danh sách các ứng viên mã.
    # Ví dụ: token_str = "[ author ]"
    candidate_map = {}
    for rel in relations:
        token = f'[ {rel} ]'
        candidate = find_relation(rel, gold_relation_map, relation_KB_map, facc1_index)
        # Nếu candidate không phải danh sách, chuyển nó thành danh sách để tạo hoán vị
        if not isinstance(candidate, list):
            candidate = [candidate]
        candidate_map[token] = candidate
        
    for ent in entities:
        token = f'[ {ent} ]'
        candidate = find_entity(ent, gold_entity_map, facc1_index)
        if not isinstance(candidate, list):
            candidate = [candidate]
        candidate_map[token] = candidate
    
    # Nếu không có token nào cần thay thế, trả về biểu thức gốc
    if not candidate_map:
        return [normed_expr]
    
    # Lấy danh sách các token và danh sách các danh sách ứng viên tương ứng
    tokens = list(candidate_map.keys())
    candidate_lists = [candidate_map[token] for token in tokens]
    
    # Tạo tất cả các hoán vị ứng viên (Cartesian product)
    all_combinations = list(product(*candidate_lists))
    
    s_expressions = []
    for comb in all_combinations:
        temp_expr = normed_expr
        # Với mỗi token, thay thế bằng ứng viên tương ứng theo hoán vị
        for token, replacement in zip(tokens, comb):
            temp_expr = temp_expr.replace(token, replacement)
        s_expressions.append(temp_expr)
    
    return s_expressions

In [13]:

def load_jsonl(file_path):
    """Đọc file JSONL và trả về danh sách các object"""


    with open(file_path, 'r', encoding='utf-8') as f:
        content = f.read().replace('(EXPECTED RESULT)', 'null').replace('(QUESTION)', 'null')

    try:
        data = ujson.loads(content)
        print("JSON loaded successfully!")
    except Exception as e:
        print(f"Error: {e}")
    return data


In [14]:
def calculate_prf1(gold_answers, pred_answers):
    """Tính Precision, Recall, F1-score"""
    if len(gold_answers) == 0:
        if len(pred_answers) == 0:
            return [1.0, 1.0, 1.0, 1]  # Đúng khi không có câu trả lời
        else:
            return [0.0, 1.0, 0.0, 1]
    elif len(pred_answers) == 0:
        return [0.0, 0.0, 0.0, 0]
    tp = 1e-40  # numerical trick

    tp = tp + len(set(gold_answers) & set(pred_answers))

    fp = len(set(pred_answers) - set(gold_answers))
    fn = len(set(gold_answers) - set(pred_answers))
    precision = tp / (tp + fp) 
    recall = tp / (tp + fn) 
    f1 = (2 * precision * recall) / (precision + recall) 
    hit = 1 if tp > 1e-40  else 0

    return [precision, recall, f1, hit]


In [15]:
# Đọc dữ liệu
predictions = load_jsonl("LLMs/beam_prediction/generated_predictions_beam02.json")
gold_data = load_jsonl("Data/LC-QuAD2.0/label_map/LC-QuAD2.0_test.json")


JSON loaded successfully!
JSON loaded successfully!


### Load danh sách quan hệ trong KB

In [16]:
file_path = "vi_property_list_file_wikidata_complete_all_mention"

relation_KB_map = {}

# Đọc file và đảo ngược quan hệ
with open(file_path, "r", encoding="utf-8") as f:
    for line in f:
        parts = line.strip().split("\t")  # Tách theo tab
        if len(parts) == 2:
            relation_KB_map[parts[1]] = parts[0]

In [17]:
import json
import time
from tqdm import tqdm

BATCH_SIZE = 100  # Số dòng mỗi batch
LOG_FILE = "progress_log.txt"

results = []
simcse_model = model  # Mô hình SimCSE của bạn
facc1_index = surface_index
parser = SExpressionParser()
start_time = time.perf_counter()  # Bắt đầu đo thời gian

# Ghi log
with open(LOG_FILE, "w", encoding="utf-8") as log_file:
    log_file.write("Bắt đầu quá trình đánh giá\n")

for i, (pred, gold) in enumerate(tqdm(zip(predictions, gold_data), total=len(predictions), desc="Đang đánh giá")):
    try:
        gold_entity_map = {v: k for k, v in gold['gold_entity_vi_map'].items()}
        gold_relation_map = {v: k for k, v in gold['gold_relation_vi_map'].items()}
        gold_answers = gold.get("answer", [])
        kq = []
        
        best_f1, best_precision, best_recall, best_hit = 0, 0, 0, 0
        lag_result = False

        for query in pred.get("predicted_query", []):
            if lag_result:
                break

            set_query = convert_normed_to_s_expression(query, gold_relation_map, gold_entity_map, relation_KB_map , facc1_index)
            for q in set_query:
                if not q:
                    continue

                sparql = parser.s_expr_to_sparql(q)
                if sparql is None:
                    continue

                query_result = execute_query_with_odbc(sparql)
                if query_result:
                    precision, recall, f1, hit = calculate_prf1(gold_answers, query_result)
                    if f1 > best_f1:
                        best_f1, best_precision, best_recall, best_hit = f1, precision, recall, hit
                    if precision == 1:
                        lag_result = True
                        break

        results.append({
            "qid": gold["question_id"],
            "answer": gold_answers,
            "result": query_result,
            "precision": best_precision,
            "recall": best_recall,
            "f1": best_f1,
            "hit": best_hit
        })

        # Ghi log tiến trình xử lý
        with open(LOG_FILE, "a", encoding="utf-8") as log_file:
            log_file.write(f"Đã xử lý xong dòng {i + 1}/{len(predictions)}\n")

        # Khi đủ 100 kết quả, lưu vào file và reset biến `results`
        if (i + 1) % BATCH_SIZE == 0 or (i + 1) == len(predictions):
            batch_id = (i + 1) // BATCH_SIZE
            filename = f"LLMs/eval_result/evaluation_results03_part_{batch_id}.json"
            with open(filename, "w", encoding="utf-8") as f:
                json.dump(results, f, indent=4)
            print(f"✅ Đã lưu {len(results)} dòng vào {filename}")
            results = []  # Reset danh sách kết quả

    except Exception as e:
        with open(LOG_FILE, "a", encoding="utf-8") as log_file:
            log_file.write(f"Lỗi tại dòng {i + 1}: {str(e)}\n")
        print(f"❌ Lỗi tại dòng {i + 1}: {e}")

# Kết thúc đo thời gian
end_time = time.perf_counter()
total_time = end_time - start_time
print(f"🎯 Quá trình đánh giá hoàn tất trong {total_time:.2f} giây!")

with open(LOG_FILE, "a", encoding="utf-8") as log_file:
    log_file.write(f"Quá trình đánh giá hoàn tất trong {total_time:.2f} giây!\n")


Đang đánh giá:   0%|          | 0/1683 [00:00<?, ?it/s]

rel: ['diễn viên lồng tiếng', 'xuất hiện trong tác phẩm']
en: ['South Park']
rel: ['diễn viên lồng tiếng', 'phỏng theo']
en: ['South Park']
rel: ['người nói', 'diễn viên lồng tiếng']
en: ['South Park']
rel: ['diễn viên lồng tiếng', 'phỏng âm']
en: ['South Park']
rel: ['người biểu diễn', 'diễn viên']
en: ['South Park', 'South Park']


Đang đánh giá:   0%|          | 1/1683 [00:17<8:17:35, 17.75s/it]

rel: ['phụ lưu', 'tọa lạc trong hoặc bên cạnh thủy vực']
en: ['Sông Winnipeg', 'Hồ Winnipeg']
rel: ['phụ lưu', 'tọa lạc trong hoặc bên cạnh thủy vực']
en: ['sông Winnipeg', 'Hồ Winnipeg']
rel: ['phụ lưu', 'vị trí']
en: ['Sông Winnipeg', 'Hồ Winnipeg']
rel: ['tiếp giáp với', 'tọa lạc trong hoặc bên cạnh thủy vực']
en: ['Sông Winnipeg', 'Hồ Winnie']


Đang đánh giá:   0%|          | 2/1683 [01:21<20:51:13, 44.66s/it]

...

rel: ['ký bởi', 'ký kết bởi']
en: ['Treaty of Versailles']
rel: ['ký kết bởi', 'người ký kết']
en: ['Hiệp ước Versailles']
rel: ['ký bởi', 'ký kết bởi']
en: ['Hiệp ước Versailles']


Đang đánh giá:  11%|█▏        | 192/1683 [11:28<1:14:21,  2.99s/it]

rel: ['ký kết bởi']
en: []
rel: []
en: []
rel: ['minimum age of compulsory education']
en: ['Oregon']
rel: ['minimum age', 'trường học']
en: ['Oregon']


Đang đánh giá:  11%|█▏        | 192/1683 [12:38<1:38:09,  3.95s/it]


KeyboardInterrupt: 

In [2]:
import glob

# Danh sách các file kết quả đánh giá
file_list = glob.glob("LLMs/eval_result/evaluation_results03_part_*.json")

# Biến để tổng hợp kết quả
total_precision = 0
total_recall = 0
total_f1 = 0
total_hits_at_1 = 0
total_samples = 0

# Đọc từng file và tổng hợp dữ liệu
for file in file_list:
    with open(file, "r", encoding="utf-8") as f:
        data = json.load(f)
        for entry in data:
            total_precision += entry.get("precision", 0)
            total_recall += entry.get("recall", 0)
            total_f1 += entry.get("f1", 0)
            total_hits_at_1 += entry.get("hit", 0)  # Hits@1 là cột "hit"
            total_samples += 1

# Tránh chia cho 0
if total_samples > 0:
    avg_precision = total_precision / total_samples
    avg_recall = total_recall / total_samples
    avg_f1 = total_f1 / total_samples
    hits_at_1 = total_hits_at_1 / total_samples
else:
    avg_precision, avg_recall, avg_f1, hits_at_1 = 0, 0, 0, 0

# Hiển thị kết quả
print(f"📊 Precision: {avg_precision:.5f}")
print(f"📊 Recall: {avg_recall:.5f}")
print(f"📊 F1-score: {avg_f1:.5f}")
print(f"📊 Hits@1: {hits_at_1:.5f}")


📊 Precision: 0.65917
📊 Recall: 0.66283
📊 F1-score: 0.65924
📊 Hits@1: 0.66394
